In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from kfsims.network import create_network, update_nodes_neighbors_cluster
from kfsims.common import init_trajectory
import numpy as np

In [3]:
traj = init_trajectory()

In [4]:
NODES = 10
ITERATIONS = 10

# No fusion

In [5]:
# no fusion
def no_fusion_step_by_step(nodes=NODES, its=ITERATIONS):
    net = create_network(nodes, iterations=its)
    msrms = {node: (i for i in node.observe()) for node in net}
    rmses = []
    for i in range(traj.X.shape[1]):
        for node, ms in msrms.items():
            m = next(ms)
            node.single_kf(m)
    for node in net:
        rmses.append(node.post_rmse(traj.X))
    return round(np.mean(rmses), 3)

In [6]:
no_fusion_step_by_step()

2.565

In [7]:
#for i in [1, 2, 5, 7, 10, 15, 20, 25, 26, 27, 28, 30, 50, 100]:
#    for no in [7, 10, 15, 20, 25, 26, 27, 28, 30, 50, 100]:
#        try:
#            print (no, i, no_fusion_step_by_step(no, i))
#        except:
#            pass
    #no_fusion_step_by_step()

In [8]:
#for i in [5, 10, 20, 50, 100]:
#    print(i, no_fusion_step_by_step(NODES, i))
#no_fusion_step_by_step()

In [9]:
def no_fusion_all_at_once():
    # all at once, should be approx. same as the above
    # just a sanity check that the the step by step works
    # as expected since it's needed in diffusion
    net = create_network(NODES, iterations=ITERATIONS)
    rmses = []
    for node in net.nodes:
        node()
        rmses.append(node.post_rmse(traj.X))
    return round(np.mean(rmses), 3)

In [10]:
no_fusion_all_at_once()

2.565

# Simple diffusion 

In [11]:
def simple_diffusion():
    net = create_network(NODES, 5, ITERATIONS)
    msrms = {node: (i for i in node.observe()) for node in net}
    rmses = []
    for i in range(traj.X.shape[1]):

        # proveď jeden KF loop pro každý node
        for node, ms in msrms.items():
            m = next(ms)
            node.single_kf(m)

        # každému nodu přiřaď průměr hyperparametrů od všech sousedů
        in_queue = set(net)
        while in_queue:
            in_queue -= update_nodes_neighbors_cluster(net, in_queue.pop())

    for node in net:
        rmses.append(node.post_rmse(traj.X))
    return round(np.mean(rmses), 3)

In [12]:
simple_diffusion()

2.35